<a href="https://colab.research.google.com/github/NiicoGonz/Alert-Room-WS/blob/master/SaludMental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

def load_gpd_world():
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    # Correcion de codigos de paises.
    world.loc[world['name'] == 'France', 'iso_a3'] = 'FRA'
    world.loc[world['name'] == 'Norway', 'iso_a3'] = 'NOR'
    world.loc[world['name'] == 'Somaliland', 'iso_a3'] = 'SOM'
    world.loc[world['name'] == 'Kosovo', 'iso_a3'] = 'RKS'

    # correcion de las dos columnas para somalia
    world = world.dissolve(by='iso_a3').reset_index()

    return world
# leyendo el archivo csv
raw_data = pd.read_csv('healtData.csv')
#Eliminamos el index que no vamos a usar
del raw_data['index']
#Correcion de la variable de años por las distintas tablas
raw_data[raw_data.Year == 'Year']
#dividimos en las subtablas

mental_health_disorder_share = raw_data.iloc[:6468].copy()
mental_health_disorder_share.columns = ['country', 'country_code', 'year','schizophrenia', 'bipolar','eating', 'anxiety','drug_use', 'depression','alcohol_use']
mental_health_disorder_share.iloc[:, 2:] = mental_health_disorder_share.iloc[:, 2:].apply(pd.to_numeric)
## Tabla 2
disorder_per_sex = raw_data.iloc[6469:54276, :6].copy()
disorder_per_sex.columns = ['country', 'country_code', 'year','males_share', 'females_share','population']
disorder_per_sex.iloc[:, 3:] = disorder_per_sex.iloc[:, 3:].apply(pd.to_numeric)
## Tabla 3
suicide_depression = raw_data.iloc[54277:102084, :6].copy()
suicide_depression.columns = ['country', 'country_code', 'year','suicide', 'depression','population']
suicide_depression.iloc[:, 3:] = suicide_depression.iloc[:, 3:].apply(pd.to_numeric)
## Tabla 4
depression_prevalence = raw_data.iloc[102085:, :4].copy()
depression_prevalence.columns = ['country', 'country_code', 'year','prevalence']
depression_prevalence.iloc[:, 3:] = depression_prevalence.iloc[:, 3:].apply(pd.to_numeric)
## Tabla auxiliar con la población global para la agregación de participaciones
population_per_year = (disorder_per_sex
                           .dropna()
                           .loc[disorder_per_sex.country_code != 'OWID_WRL',['country_code', 'year', 'population']]
                           .copy()
                      )
population_per_year['year'] = pd.to_numeric(population_per_year['year'])
population_per_year['population'] = pd.to_numeric(population_per_year['population'])
# Exportar DataFrames a un archivo Excel
with pd.ExcelWriter('datos_salud_mental.xlsx') as writer:
    mental_health_disorder_share.to_excel(writer, sheet_name='Salud Mental')
    disorder_per_sex.to_excel(writer, sheet_name='Trastorno por Sexo')
    suicide_depression.to_excel(writer, sheet_name='Suicidio y Depresión')
    depression_prevalence.to_excel(writer, sheet_name='Prevalencia de Depresión')
    population_per_year.to_excel(writer, sheet_name='Población por Año')


<ipython-input-5-96c0bca0ab55>:22: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('healtData.csv')
<ipython-input-5-96c0bca0ab55>:28: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  mental_health_disorder_share.iloc[:, 2:] = mental_health_disorder_share.iloc[:, 2:].apply(pd.to_numeric)
